In [1]:
import cv2
import os
import csv
import pandas as pd
import numpy as np
from PIL import Image 
import random

In [2]:
path = "/Users/denniscimorosi/Desktop/Tesi/Logos/logos/"
output_file = path + "states.csv"

In [3]:
logos = os.listdir(path)
n = 5 #1000 per 10000 immagini
noise = 70

In [4]:
class LogoBinarizer:
    def _img_preprocess(self, img: np.array):
        n_channels = img.shape[2]
        if n_channels == 4:
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
        elif n_channels == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        return img

    def _otsu(self, img, mask=[]):
        least_var = float('inf')
        final_thr = None
        image = img.copy()

        is_gray = (len(img.shape) == 2)
        if not is_gray:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        masked_image = image if len(mask) <= 0 else image[mask]

        # useful for retrieving occurrences
        [hist, _] = np.histogram(masked_image, bins=256, range=(0, 255))

        p_all = masked_image.size
        for t in range(256):
            # number of pixels in each class
            n_bg = np.sum(hist[:t])
            n_fg = np.sum(hist[t:])

            if n_bg < 1 or n_fg < 1: continue

            # proportion of pixels in each class, sum up to 1
            w_bg = n_bg / p_all
            w_fg = n_fg / p_all

            # get values in each class,
            # obtained by the product of the value of the pixels by their occurrences
            vals_bg = np.array(range(t)) * hist[:t]
            vals_fg = np.array(range(t, 256)) * hist[t:]

            mean_bg = np.sum(vals_bg) / n_bg
            mean_fg = np.sum(vals_fg) / n_fg

            var_bg = np.sum(np.power(vals_bg - mean_bg, 2)) / n_bg
            var_fg = np.sum(np.power(vals_fg - mean_fg, 2)) / n_fg

            # σ^2(t)=ωbg(t)σ^2bg(t)+ωfg(t)σ^2fg(t)
            total_var = w_bg * var_bg + w_fg * var_fg

            if total_var < least_var:
                least_var = total_var
                final_thr = t

        # set new colors based on obtained threshold
        if final_thr:
            image[image <= final_thr] = 0
            image[image > final_thr] = 255

        return image

    def _get_closed_contours(self, img_bin: np.array):
        edges = cv2.Canny(img_bin, 0, 255)
        contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # if a polygon has a child (x[2] != -1) it is considered to be closed
        closed_idxs = [i for i, x in enumerate(hierarchy[0]) if x[2] != -1]
        closed_contours = [contours[i] for i in closed_idxs]

        return closed_contours

    # unusued, could come in handy in the future
    def _is_roi_balanced(self, img_bin, roi_mask):
        pixels = img_bin[roi_mask]
        whites = pixels[pixels == 255].size
        white_prop = whites / pixels.size
        black_prop = 1 - white_prop
        return black_prop < 0.8 and white_prop < 0.8

    def _find_rois(self, img_bin: np.array, contours):
        inner_rois = []
        for contour in contours:
            roi_mask = self._mask_from_contour(img_bin, contour)
            roi = img_bin[roi_mask]
            inner_rois.append((roi_mask, contour))
        return inner_rois

    def _mask_from_contour(self, img_bin: np.array, contour):
        blank = np.zeros_like(img_bin)
        cv2.fillPoly(blank, [contour], 255)
        roi_mask = (blank == 255)
        return roi_mask

    def binarize(self, img: np.array):
        img = self._img_preprocess(img)
        img_bin = self._otsu(img)
        contours = self._get_closed_contours(img_bin)
        rois = self._find_rois(img_bin, contours)

        for roi_mask, contour in rois:
            roi_bin = self._otsu(img, roi_mask)
            img_bin[roi_mask] = roi_bin[roi_mask]

        return img_bin

In [5]:
class CapcthaStarGenerator:
    def __init__(self, sensibility=7, noise=30, tile_dim=5, n_tile_h=50, n_tile_w=50,
                 overlap_noise=True, separate_movements=False,
                 disable_big=True, enable_rotation=False):

        self.sensibility = sensibility
        self.noise = noise

        self.tile_dim = tile_dim
        self.h = self.tile_dim * n_tile_h
        self.w = self.tile_dim * n_tile_w

        # da vedere coi profs
        self.overlap_noise = overlap_noise
        self.separate_movements = separate_movements
        self.disable_big = disable_big
        self.enable_rotation = enable_rotation
        self.binarizer = LogoBinarizer()

    # TODO add rotation here
    def _preprocess(self, img: np.array):
        img = self.binarizer.binarize(img)
        fg_mask = self._get_fg_mask(img, (self.h, self.w))

        img = cv2.resize(img, (self.h, self.w), interpolation=cv2.INTER_NEAREST)

        return img, fg_mask

    # returns the mask corresponding to the foreground of 'img' resized according the 'size' parameter
    def _get_fg_mask(self, img: np.array, size: tuple):
        edges = cv2.Canny(img, 0, 255)

        # if a polygon has a child (x[2] != -1) it is considered to be closed
        # closed_idxs = [i for i, x in enumerate(hierarchy[0]) if x[2] != -1]
        # closed_contours = [contours[i] for i in closed_idxs]

        # prove
        kernel = np.ones((3, 3))
        edges = cv2.dilate(edges, kernel, iterations=32)
        edges = cv2.erode(edges, kernel, iterations=30)
        contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # retrive the biggest contour
        contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
        biggest_contour = max(contour_sizes, key=lambda x: x[0])[1]

        # to obtain the mask of right size:
        # draw the contour in white on a black image, resize it, and get a mask from this binary image
        blank = np.zeros_like(img)
        cv2.fillPoly(blank, [biggest_contour], 255)
        blank = cv2.resize(blank, size, interpolation=cv2.INTER_NEAREST)
        fg_mask = (blank == 255)

        return fg_mask

    def _adjust_coordinates(self, x, sums, threshold):
        new_x = x
        if sums[1] <= threshold:
            new_x -= 2
        elif sums[0] <= threshold:
            new_x -= 1
        if sums[self.tile_dim - 2] <= threshold:
            new_x += 2
        elif sums[self.tile_dim - 1] <= threshold:
            new_x += 1
        return new_x

    def _get_tile_from_coordinates(self, x, y, mask):
        # get range for x coordinate of corresponding tile
        x_tile = int(np.floor(x / self.tile_dim))
        x_tile_range_lower = self.tile_dim * x_tile
        x_tile_range_upper = x_tile_range_lower + self.tile_dim

        # get range for y coordinate of corresponding tile
        y_tile = int(np.floor(y / self.tile_dim))
        y_tile_range_lower = self.tile_dim * y_tile
        y_tile_range_upper = y_tile_range_lower + self.tile_dim

        tile = mask[y_tile_range_lower:y_tile_range_upper,
               x_tile_range_lower:x_tile_range_upper]
        return tile

    def _count_non_transparent_in_tiles(self, mask):
        slicesx = np.arange(0, mask.shape[0], self.tile_dim)
        pixel_count = np.add.reduceat(mask, slicesx, axis=0)
        slicesy = np.arange(0, mask.shape[1], self.tile_dim)
        pixel_count = np.add.reduceat(pixel_count, slicesy, axis=1)
        return pixel_count

    def _condition_1(self, pixel_count):
        tile_area = self.tile_dim ** 2
        xs, ys = np.where(pixel_count == tile_area)
        # map each tile to the central pixel of the original image
        xs = xs * self.tile_dim + (np.ceil(self.tile_dim / 2))
        ys = ys * self.tile_dim + (np.ceil(self.tile_dim / 2))
        coordinates = list(zip(xs, ys))
        return np.array(coordinates)

    def _condition_2(self, pixel_count, mask):
        tile_area = self.tile_dim ** 2
        min_val = tile_area / 3  # nostro:8

        # in this condition we are excluding the pixels that would satisfy condtion 1
        # (da cambiare?)
        xs, ys = np.where((pixel_count > min_val) & (pixel_count < tile_area))

        # map each tile to the central pixel of the original image
        xs = xs * self.tile_dim + (np.ceil(self.tile_dim / 2))
        ys = ys * self.tile_dim + (np.ceil(self.tile_dim / 2))

        # depending on where the transparent pixels are in the tile,
        # move the coordinates
        to_adjust_coordinates = np.array(list(zip(xs, ys)))
        threshold = self.tile_dim / 3

        coordinates = []
        # for each point to adjust
        for (x, y) in to_adjust_coordinates:  # TODO: vectorize
            # retrieve the corresponding tile and sums over cols and rows
            tile = self._get_tile_from_coordinates(x, y, mask)
            col_sums = tile.sum(axis=0)
            row_sums = tile.sum(axis=1)

            # adjust the coordinates based on the row and col sums
            new_x = self._adjust_coordinates(x, row_sums, threshold)
            new_y = self._adjust_coordinates(y, col_sums, threshold)
            coordinates.append((new_x, new_y))
        return np.array(coordinates)

    def _get_c(self, val, sol_x, sol_y, offset):
        m1 = random.randint(-10000 * self.sensibility, 10000 * self.sensibility) / 100000
        m2 = random.randint(-10000 * self.sensibility, 10000 * self.sensibility) / 100000
        c = round(val - sol_y * m1 - sol_x * m2) + offset
        return m2, m1, int(c)

    def _to_riki_string(self, cs):
        final_riki = ""
        for t in cs:
            riki_string = "@{} {} {} {} {} {}".format(int(t[0] * 100000), int(t[1] * 100000), t[2], int(t[3] * 100000),
                                                      int(t[4] * 100000), t[5])
            final_riki += riki_string
        return final_riki

    # TODO aggiungere maschera
    def _picture_decomposition(self, img_bin: np.array, fg_mask=[]):
        if len(fg_mask) < 1:
            fg_mask = np.full_like(img_bin, True)

        # binarize (False => trasparent)
        mask = np.logical_and(img_bin > 0, fg_mask)  # mask = np.array(img[fg_mask]) > 200

        # if the non-transparent pixels are more than the half,
        # invert binary mask
        if mask.sum() > mask.size / 2: mask = mask ^ 1

        tile_area = self.tile_dim ** 2

        # count non-transparent pixels in each tile
        pixel_count = self._count_non_transparent_in_tiles(mask)

        # condition 1 => filled with non-transparent pixels (i.e. 1s)
        # mapped at the center of the tile
        coordinates_1 = self._condition_1(pixel_count)

        # condition 2 => above min_val & adjust based on tile distribution
        coordinates_2 = self._condition_2(pixel_count, mask)

        coordinates = np.concatenate((coordinates_1, coordinates_2))

        return coordinates

    def _trajectory_computation(self, coordinates):
        # calculate cs
        offset_x = random.randint(0, 300 - self.w)
        offset_y = random.randint(0, 300 - self.h)
        sol_x = random.randint(10, 290)
        sol_y = random.randint(10, 290)
        cs = [self._get_c(x, sol_x, sol_y, offset_x) +
              self._get_c(y, sol_x, sol_y, offset_y) for x, y in coordinates]

        # add noise
        n_noise = int(coordinates.shape[0] * self.noise / 100)
        noise_coordinates = [(random.randint(0, 300), random.randint(0, 300)) for _ in range(n_noise)]
        cs_noise = [self._get_c(x, random.randint(0, 300), random.randint(0, 300), 0) +
                    self._get_c(y, random.randint(0, 300), random.randint(0, 300), 0) for x, y in noise_coordinates]
        cs = cs + cs_noise
        return cs, sol_x, sol_y

    def generate_captcha(self, img):
        img, fg_mask = self._preprocess(img)
        coordinates = self._picture_decomposition(img)  # for now, we do not use the foreground mask
        cs, sol_x, sol_y = self._trajectory_computation(coordinates)
        stars = self._to_riki_string(cs)
        return stars, sol_x, sol_y

In [6]:
def create_logo_dataset(logos, noise, n):
    csg = CapcthaStarGenerator(noise=noise)
    df = pd.DataFrame(columns=['logo', 'solx', 'soly', 'stars'])
    for logo in logos:
        img = Image.open(path + logo)
        img = np.array(img)
        for _ in range(n):
            stars, sol_x, sol_y = csg.generate_captcha(img)
            row = pd.DataFrame([[logo, sol_x, sol_y,stars]], columns=['logo', 'solx', 'soly', 'stars'])
            df = pd.concat((df, row))
    return df
        

df = create_logo_dataset(logos, noise, n)

In [7]:
union_path = "/Users/denniscimorosi/Desktop/Tesi/Logos/union/"

In [8]:
def generate_neg(sol_x, sol_y, tolerance=4):
    to_exclude_x = range(sol_x-tolerance,sol_x+tolerance+1)
    pos_x = random.choice([i for i in range(10,290) if i not in to_exclude_x])
    to_exclude_y = range(sol_y-tolerance,sol_y+tolerance+1)
    pos_y = random.choice([i for i in range(10,290) if i not in to_exclude_y])
    return pos_x, pos_y

def generate_pos(sol_x, sol_y, tolerance=4):
    shift_x = random.randint(-tolerance,tolerance)
    shift_y = random.randint(-tolerance,tolerance)
    pos_x = sol_x + shift_x
    pos_y = sol_y + shift_y
    return pos_x, pos_y

In [9]:
class Point:
    def __init__(self, mxx, mxy, cx, myx, myy, cy):
        self.mxx = int(mxx)
        self.mxy = int(mxy)
        self.cx = int(cx)
        self.myx = int(myx)
        self.myy = int(myy)
        self.cy = int(cy)

class CaptchaStarDrawer:
  def __init__(self, point_size=4):
    self.point_size = point_size
    self.stars = []

  def _parse_lines(self, lines):
    res = []
    points = lines.split('@')
    for p in points:
      split = p.split(" ")
      if len(split) != 6: continue
      mxx, mxy, cx, myx, myy, cy = p.split(" ")
      new_point = Point(mxx, mxy, cx, myx, myy, cy)
      res.append(new_point)
    return res

  def get_state_as_img(self, riki_string, pos_x, pos_y):
    points = self._parse_lines(riki_string)
    pos_x /= 100000
    pos_y /= 100000

    img = np.zeros((300,300))
    for p in points:
      # x=stars[i].mxy*mousey+stars[i].mxx*mousex+stars[i].cx-POINT_SIZE/2;
      x = int(round(p.mxy*pos_y+p.mxx*pos_x+p.cx-self.point_size/2))
      # y=stars[i].myx*mousex+stars[i].myy*mousey+stars[i].cy-POINT_SIZE/2;
      y = int(round(p.myx*pos_x+p.myy*pos_y+p.cy-self.point_size/2))

      start_x = int(round(x-(self.point_size/2)))
      end_x = int(round(x+(self.point_size/2)))
      start_y = int(round(y-(self.point_size/2)))
      end_y = int(round(y+(self.point_size/2)))
      if end_x < 300 and start_x > -1:
        if end_y < 300 and start_y > -1:
          img[start_x:end_x, start_y:end_y] = 255

    img = Image.fromarray(img)
    img = img.convert('RGB')
    return img

In [10]:
csd = CaptchaStarDrawer()

In [11]:
union_list = []

In [12]:
def create_pos_sample(row):
    global union_list
    for _ in range(1): #5
        pos_x, pos_y = generate_pos(row['solx'], row['soly'])
        img = csd.get_state_as_img(row['stars'], pos_x, pos_y)
        logo_name = row['logo'].split('.')[0]
        logo_name = logo_name + str(pos_x) + str(pos_y) + "-pos" + ".png"
        union_list.append(logo_name)
        img.save(union_path + logo_name)
        
def create_neg_sample(row):
    global union_lis
    for _ in range(1):#5
        pos_x, pos_y = generate_neg(row['solx'], row['soly'])
        img = csd.get_state_as_img(row['stars'], pos_x, pos_y)
        logo_name = row['logo'].split('.')[0]
        logo_name = logo_name + str(pos_x) + str(pos_y) + "-neg" + ".png"
        union_list.append(logo_name)
        img.save(union_path + logo_name)

In [13]:
#svuota union folder
!rm -rf union/*

zsh:1: argument list too long: rm


In [14]:
df.apply(create_pos_sample, axis=1)
first_negative = len(union_list)
df.apply(create_neg_sample, axis=1)

0    None
0    None
0    None
0    None
0    None
     ... 
0    None
0    None
0    None
0    None
0    None
Length: 2000, dtype: object

In [15]:
union_path = "/Users/denniscimorosi/Desktop/Tesi/Logos/union/"
output_file = "/Users/denniscimorosi/Desktop/Tesi/Logos/states.csv"

In [16]:
df2 = pd.DataFrame(columns=['captcha', 'label'])

In [17]:
for i,captcha in enumerate(union_list):
    label = (i < first_negative)
    row = pd.DataFrame([[captcha, label]], columns=['captcha', 'label'])
    df2 = pd.concat((df2, row))
    if i%20000 == 0:
        print(i)

0


In [18]:
df2.to_csv(output_file, index=False)